In [ ]:
# For colab
!pip uninstall tensorflow -y
!pip install tensorflow==2.3.1 tensorflow-quantum
!rm -rf quantum-gans
!git clone https://github.com/WiktorJ/quantum-gans
!cd quantum-gans; pip install .

In [5]:
from typing import Any

import sympy
import math
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import numpy as np
from cirq import GridQubit, ops
from qsgenerator.utils import map_to_radians
from qsgenerator.qugans import circuits
from qsgenerator.qugans.training import Trainer, CustomSchedule
from qsgenerator.phase.circuits import build_ground_state_circuit
from qsgenerator.phase.analitical import construct_hamiltonian, get_ground_state_for_g, get_theta_v, get_theta_w, get_theta_r

In [7]:
generator_layers = 2
discriminator_layers = 2
data_bus_size = 3

In [8]:
gen, gs, disc, ds, ls, data_qubits, out_qubit = circuits.build_gan_circuits(generator_layers, discriminator_layers, data_bus_size)

In [9]:
real, real_symbols = build_ground_state_circuit(qubits=data_qubits)

In [10]:
pure_gen = gen.copy()
gen.append([disc])

In [11]:
pure_real = real.copy()
real.append([disc])

In [7]:
print("REAL GROUND STATE: ", pure_real)

REAL GROUND STATE:  (0, 0): ───H───@─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
               │
(0, 1): ───────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────Ry(theta_w)───X───X───Ry(theta_w)───X───Ry(theta_v)───X───X───Ry(theta_v)───X───
               │                                                                                                                                                                                                   │                                       │
(0, 2): ───────X───Z───Ry(theta_r)───X─────────────@───X───────────────────────────────────@───X──────────────────────────────────────────────────────────

In [8]:
print("GENERATOR:", pure_gen)

GENERATOR: (0, 2): ───Rx(g0)───Rz(g4)───ZZ────────────────────────Rx(g11)───Rz(g15)───ZZ──────────────────────────
                             │                                             │
(0, 3): ───Rx(g1)───Rz(g5)───ZZ^g8────────────ZZ───────Rx(g12)───Rz(g16)───ZZ^g19─────────────ZZ───────
                                              │                                               │
(0, 4): ───Rx(g2)───Rz(g6)────────────ZZ──────ZZ^g10───Rx(g13)───Rz(g17)─────────────ZZ───────ZZ^g21───
                                      │                                              │
(0, 5): ───Rz(l)────Rx(g3)───Rz(g7)───ZZ^g9────────────Rz(l)─────Rx(g14)───Rz(g18)───ZZ^g20────────────


In [9]:
print("DISCRIMINATOR:", disc)

DISCRIMINATOR: (0, 0): ───Rx(d0)───Rz(d5)────────────ZZ────────────────Rx(d14)───Rz(d19)─────────────ZZ────────────────
                                      │                                               │
(0, 1): ───Rz(l)────Rx(d1)───Rz(d6)───ZZ^d10───ZZ───────Rz(l)─────Rx(d15)───Rz(d20)───ZZ^d24───ZZ───────
                                               │                                               │
(0, 2): ───Rx(d2)───Rz(d7)───ZZ────────────────ZZ^d12───Rx(d16)───Rz(d21)───ZZ─────────────────ZZ^d26───
                             │                                              │
(0, 3): ───Rx(d3)───Rz(d8)───ZZ^d11───ZZ────────────────Rx(d17)───Rz(d22)───ZZ^d25────ZZ────────────────
                                      │                                               │
(0, 4): ───Rx(d4)───Rz(d9)────────────ZZ^d13────────────Rx(d18)───Rz(d23)─────────────ZZ^d27────────────


In [10]:
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 21) + \
                   np.random.normal(scale=eps, size=(22,))
init_disc_weights = np.random.normal(size=(28,))

gen_weights = tf.Variable(init_gen_weights, dtype=tf.float32)
disc_weights = tf.Variable(init_disc_weights, dtype=tf.float32)

In [11]:
learning_rate = CustomSchedule()

opt = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [12]:
g_provider = lambda: 0

In [13]:
trainer = Trainer(g_provider, 
                  data_bus_size, 
                  disc, 
                  gen, 
                  real, 
                  out_qubit, 
                  ds, 
                  gs, 
                  real_symbols, 
                  ls,
                  use_analytical_expectation=True)

In [ ]:
epochs = 50
disc_iteration = 100
gen_iteration = 2
trainer.train(disc_weights,
      gen_weights, 
      opt, 
      epochs=epochs, 
      disc_iteration=disc_iteration, 
      gen_iteration=gen_iteration)

Instructions for updating:
Use fn_output_signature instead
Epoch 0: cost = [[0.2982991]]
Prob(real classified as real):  [[0.4976718]]
Prob(fake classified as real):  [[0.7959709]]
Epoch 0: cost = [[-0.7959709]]
Prob(fake classified as real):  [[0.7959709]]
Discriminator cost:  [[0.2982991]]
Generator weights: <tf.Variable 'Variable:0' shape=(22,) dtype=float32, numpy=
array([-10.1538925,  -5.8757625,  -5.8703175,   3.001207 , -13.019004 ,
       -12.693356 ,   1.9784341, -13.137627 , -13.786987 , -13.515038 ,
       -13.311686 , -12.854742 ,   2.3864002,   5.4604774,  -5.8757343,
       -12.866825 , -13.503126 ,  -5.8818154,  -5.876974 ,  -5.8876247,
       -13.543596 , -13.511531 ], dtype=float32)>
Discriminator weights <tf.Variable 'Variable:0' shape=(28,) dtype=float32, numpy=
array([  28.310925  ,   -1.3881841 ,   -0.61370814, -102.06886   ,
        -83.644295  ,  -14.386199  ,   50.432297  ,   52.792294  ,
         82.30965   ,  -98.54453   ,   94.65001   ,   32.676636  ,
       

In [16]:
g = 0
size = 3
ground_state = get_ground_state_for_g(g, size)
rad = map_to_radians(g)


trained_disc_weights = tf.Variable(np.array([-146.08809  ,   47.122505 ,    2.0412364, -251.7854   ,
        -70.20249  , -237.18315  ,  288.8051   ,  145.9316   ,
         39.44586  ,  -33.36509  ,  167.49129  ,   15.891043 ,
       -352.99362  , -159.65291  ,   70.686714 ,  145.67868  ,
       -141.0045   ,   27.411167 ,  170.49501  ,   86.810104 ,
        190.26714  ,  -94.70031  , -282.55457  ,  262.3764   ,
        210.5744   , -130.14821  ,   37.32039  , -442.34482  ]), dtype=tf.float32)

trained_gen_weights = tf.Variable(np.array([  76.99695 ,   90.76301 ,  179.41345 ,  -22.535416,   85.21917 ,
       -148.49991 ,   35.802727,  -51.09742 ,    8.920626,   87.83092 ,
         94.84478 ,  -59.440678,   58.927383,  131.01495 ,   26.18327 ,
         55.878807,   33.834774,  -42.0028  ,   49.96799 ,  -54.968147,
        -81.59106 ,  -55.935013, rad]), dtype=tf.float32)



state_vector = tfq.layers.State()(pure_gen, symbol_names=gs + (ls,), symbol_values=tf.reshape(trained_gen_weights, (1, trained_gen_weights.shape[0]))).values.numpy()#%%

traced_out_state = cirq.partial_trace_of_state_vector_as_mixture(state_vector, [0,1,2])

In [17]:
prob, highest_prob_state = max(traced_out_state)
print(f"Max prob state has prob: {prob}")

Max prob state has prob: 0.8409811854362488


In [18]:
highest_prob_state

array([-0.3339611 +0.j        , -0.4318884 +0.53677016j,
       -0.04706756-0.07611239j, -0.13058527+0.04029261j,
        0.24577042+0.09136288j,  0.46661744-0.2706012j ,
        0.0440205 +0.07026347j,  0.12984814-0.06077267j], dtype=complex64)

In [20]:
get_ground_state_for_g(g, size)

array([1., 0., 0., 0., 0., 0., 0., 0.])

In [21]:
np.arccos(cirq.fidelity(highest_prob_state, get_ground_state_for_g(g,size)))

1.4590337876925663

In [22]:
cirq.fidelity(highest_prob_state, get_ground_state_for_g(g, size))

0.11153001590324951